# ComfyUI API

Currently, there doesn't seem to exist any good documentation for the ComfyUI API.  
ComfyUI is Python web UI that uses Stable Diffusion XL to generate images.  

Luckily, the project is open-source so we can peek inside and document it ourselves.

### List of available routes in comfyui\ComfyUI\server.py:

GET methods:

- / - Root handler
- /ws - Websocket handler
- /embeddings - Get embeddings
- /extensions - Get extensions
- /upload/image - Upload image
- /upload/mask - Upload mask
- /view - View image

example:
http://127.0.0.1:8188/view?filename=ComfyUI_00012_.png&preview=true

- /view_metadata/{folder_name} - View metadata
- /system_stats - Get system stats
- /prompt - Get prompt
- /object_info - Get object info
- /object_info/{node_class} - Get object info for a specific node class
- /history - Get history
- /history/{prompt_id} - Get history for a specific prompt id
- /queue - Get queue

POST methods:

- /prompt - Post prompt
- /queue - Post queue
- /interrupt - Post interrupt
- /history - Post history

### How to access comfyUI on the local network?
It's super easy, modify run_nvidia_gpu.bat and add **--listen** at the end of the first line

**GET method**

In [116]:
import requests
import json

def get_request(endpoint):
    url = f"http://192.168.1.220:8188/{endpoint}"
    response = requests.get(url)
    if response.text:
        data = json.loads(response.text)
        return data
    else:
        print("No data received")

In [117]:
# test by using GET queue
get_request("queue")

{'queue_running': [], 'queue_pending': []}

**POST method**

In [118]:
import json
from urllib import request, parse
import random

def queue_prompt(prompt_workflow):
    p = {"prompt": prompt_workflow}
    data = json.dumps(p).encode('utf-8')
    req =  request.Request("http://127.0.0.1:8188/prompt", data=data)
    res = request.urlopen(req)
    return res.read().decode('utf-8')

prompt_workflow = json.load(open('workflow_api.json'))

# randomize seed
prompt_workflow["3"]["inputs"]["seed"] = random.randint(1,18446744073709551614) 

# queue the prompt, save the prompt_id
response = queue_prompt(prompt_workflow)
parsed_json = json.loads(response)
prompt_id = parsed_json['prompt_id']

print(prompt_id)


b013f9c3-ffdf-4be5-bc89-997e26f74e85


In [121]:
from IPython.display import display, Markdown



# get filename for my prompt
response = get_request("history")
filename = response[prompt_id]['outputs']['9']['images'][0]['filename']

display(Markdown(f"The filename is: **{filename}**"))
print(filename)

ComfyUI_00190_.png
